In [1]:
def load_data():
    """
    Función para cargar y preparar los datos del dataset auto_mpg.
    
    Esta función realiza las siguientes operaciones:
    1. Carga el dataset desde un archivo CSV
    2. Limpia los datos eliminando valores nulos
    3. Mapea valores numéricos de origen a nombres de países
    4. Separa la variable objetivo (MPG) de las características
    
    Returns:
        tuple: (x, y) donde x son las características y y es la variable objetivo
    """
    import pandas as pd

    # Cargar el dataset de consumo de combustible de automóviles desde el archivo CSV
    dataset = pd.read_csv("../files/input/auto_mpg.csv")
    
    # Eliminar todas las filas que contengan valores nulos (NaN)
    # Esto es importante para evitar errores en el entrenamiento del modelo
    dataset = dataset.dropna()
    
    # Mapear los valores numéricos de la columna "Origin" a nombres de países
    # 1 = USA, 2 = Europe, 3 = Japan
    # Esto convierte una variable numérica en categórica más interpretable
    dataset["Origin"] = dataset["Origin"].map(
        {1: "USA", 2: "Europe", 3: "Japan"},
    )
    
    # Extraer la variable objetivo (MPG - Miles Per Gallon)
    # pop() elimina la columna del dataset y la retorna
    y = dataset.pop("MPG")
    
    # Crear una copia del dataset sin la variable objetivo
    # Estas serán nuestras características (features) para el modelo
    x = dataset.copy()

    return x, y

In [2]:
def make_train_test_split(x, y):
    """
    Función para dividir los datos en conjuntos de entrenamiento y prueba.
    
    Esta función separa los datos en dos conjuntos:
    - Entrenamiento (75%): para entrenar el modelo
    - Prueba (25%): para evaluar el rendimiento del modelo
    
    Args:
        x: Características (features)
        y: Variable objetivo (target)
    
    Returns:
        tuple: (x_train, x_test, y_train, y_test) conjuntos de entrenamiento y prueba
    """
    from sklearn.model_selection import train_test_split

    # Dividir los datos en conjuntos de entrenamiento (75%) y prueba (25%)
    # test_size=0.25 significa que el 25% de los datos se usarán para prueba
    # random_state=123456 asegura que la división sea reproducible
    # (siempre obtendremos la misma división aleatoria)
    (x_train, x_test, y_train, y_test) = train_test_split(
        x,                    # Características de entrada
        y,                    # Variable objetivo
        test_size=0.25,       # 25% para prueba, 75% para entrenamiento
        random_state=123456,  # Semilla para reproducibilidad
    )
    return x_train, x_test, y_train, y_test

In [3]:
def make_pipeline(estimator):
    """
    Función para crear un pipeline de procesamiento de datos y modelado.
    
    El pipeline consta de tres pasos principales:
    1. Transformación de datos (codificación y escalado)
    2. Selección de características (SelectKBest)
    3. Modelo de predicción (estimador)
    
    Args:
        estimator: El modelo de machine learning a usar (ej: LinearRegression, MLPRegressor)
    
    Returns:
        Pipeline: Pipeline completo listo para entrenar
    """
    from sklearn.compose import ColumnTransformer
    from sklearn.feature_selection import SelectKBest, f_regression
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import OneHotEncoder, StandardScaler

    # PASO 1: Configurar el transformador de columnas
    # Esto maneja diferentes tipos de datos de manera apropiada
    transformer = ColumnTransformer(
        transformers=[
            # OneHotEncoder para la variable categórica "Origin"
            # dtype="int" convierte los valores booleanos a enteros (0, 1)
            # Esto crea variables dummy para USA, Europe, Japan
            ("ohe", OneHotEncoder(dtype="int"), ["Origin"]),
        ],
        # remainder=StandardScaler() aplica normalización estándar 
        # a todas las demás columnas numéricas
        # Esto centra los datos en 0 y los escala a desviación estándar de 1
        remainder=StandardScaler(),
    )

    # PASO 2: Configurar la selección de características
    # SelectKBest selecciona las k mejores características basándose en una función de puntuación
    # f_regression es la función de puntuación para problemas de regresión
    # Evalúa la correlación lineal entre cada característica y la variable objetivo
    selectkbest = SelectKBest(score_func=f_regression)

    # PASO 3: Crear el pipeline completo
    # Los pasos se ejecutan secuencialmente en el orden especificado
    pipeline = Pipeline(
        steps=[
            # 1. Transformar datos (codificación + normalización)
            ("tranformer", transformer),
            # 2. Seleccionar las mejores características
            ("selectkbest", selectkbest),
            # 3. Aplicar el modelo de predicción
            ("estimator", estimator),
        ],
        verbose=False,  # No mostrar información detallada durante la ejecución
    )

    return pipeline

In [4]:
def make_grid_search(estimator, param_grid, cv=5):
    """
    Función para crear un GridSearchCV que encuentra los mejores hiperparámetros.
    
    GridSearchCV prueba todas las combinaciones posibles de parámetros
    especificados en param_grid y selecciona la mejor combinación
    basándose en validación cruzada.
    
    Args:
        estimator: El pipeline o modelo a optimizar
        param_grid: Diccionario con los parámetros a probar
        cv: Número de folds para validación cruzada (default=5)
    
    Returns:
        GridSearchCV: Objeto configurado para búsqueda de hiperparámetros
    """
    from sklearn.model_selection import GridSearchCV

    # Crear el objeto GridSearchCV para búsqueda exhaustiva de hiperparámetros
    grid_search = GridSearchCV(
        estimator=estimator,           # El pipeline/modelo a optimizar
        param_grid=param_grid,         # Diccionario con parámetros a probar
        cv=cv,                         # Validación cruzada de 5 folds por defecto
        # Métrica de evaluación: Error Absoluto Medio negativo
        # Se usa negativo porque GridSearch busca maximizar, pero queremos minimizar el error
        scoring='neg_mean_absolute_error',
    )

    return grid_search

In [5]:
def save_estimator(estimator):
    """
    Función para guardar un modelo entrenado en disco.
    
    Utiliza pickle para serializar el objeto del modelo y guardarlo
    en un archivo binario. Esto permite reutilizar el modelo entrenado
    sin necesidad de volver a entrenarlo.
    
    Args:
        estimator: El modelo entrenado a guardar
    """
    import pickle

    # Abrir archivo en modo binario de escritura ("wb")
    # El modelo se serializa usando pickle y se guarda como "estimator.pickle"
    with open("estimator.pickle", "wb") as file:
        # pickle.dump() serializa el objeto estimator y lo escribe al archivo
        # Esto preserva completamente el estado del modelo entrenado
        pickle.dump(estimator, file)

In [6]:
def load_estimator():
    """
    Función para cargar un modelo previamente guardado desde disco.
    
    Busca el archivo "estimator.pickle" y lo carga si existe.
    Si no existe, retorna None.
    
    Returns:
        estimator or None: El modelo cargado o None si no existe el archivo
    """
    import os
    import pickle

    # Verificar si existe el archivo del modelo guardado
    # Si no existe, significa que no hay un modelo previo guardado
    if not os.path.exists("estimator.pickle"):
        return None
    
    # Abrir el archivo en modo binario de lectura ("rb")
    with open("estimator.pickle", "rb") as file:
        # pickle.load() deserializa el objeto desde el archivo
        # Esto restaura completamente el modelo con todos sus parámetros entrenados
        estimator = pickle.load(file)

    return estimator

In [7]:
def train_linear_regression():
    """
    Función principal para entrenar un modelo de regresión lineal.
    
    Esta función implementa un flujo completo de machine learning:
    1. Carga los datos
    2. Divide en entrenamiento/prueba
    3. Crea pipeline con selección de características
    4. Busca los mejores hiperparámetros
    5. Compara con modelo previo guardado
    6. Guarda el mejor modelo
    """
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error
    

    # PASO 1: Cargar y preparar los datos
    data, target = load_data()

    # PASO 2: Dividir los datos en conjuntos de entrenamiento y prueba
    x_train, x_test, y_train, y_test = make_train_test_split(
        x=data,
        y=target,
    )

    # PASO 3: Crear el pipeline con LinearRegression como estimador
    # El pipeline incluye: transformación -> selección de características -> regresión lineal
    pipeline = make_pipeline(
        estimator=LinearRegression(),
    )

    # PASO 4: Configurar búsqueda de hiperparámetros
    # Probará diferentes valores de k (número de características a seleccionar)
    # desde 1 hasta el total de características disponibles
    estimator = make_grid_search(
        estimator=pipeline,
        param_grid={
            # Probar todos los valores posibles de k para SelectKBest
            "selectkbest__k": range(1, len(x_train.columns) + 1),
        },
        cv=5,  # Validación cruzada de 5 folds
    )

    # PASO 5: Entrenar el modelo con búsqueda de hiperparámetros
    # GridSearchCV probará todas las combinaciones y encontrará la mejor
    estimator.fit(x_train, y_train)

    # PASO 6: Comparar con modelo previamente guardado (si existe)
    best_estimator = load_estimator()

    if best_estimator is not None:
        # Calcular el error del modelo guardado previamente
        saved_mae = mean_absolute_error(
            y_true=y_test, y_pred=best_estimator.predict(x_test)
        )

        # Calcular el error del modelo recién entrenado
        current_mae = mean_absolute_error(
            y_true=y_test, y_pred=estimator.predict(x_test)
        )

        # Si el modelo guardado es mejor, usarlo en lugar del nuevo
        # Esto implementa una forma de "early stopping" o conservación del mejor modelo
        if saved_mae < current_mae:
            estimator = best_estimator

    # PASO 7: Guardar el mejor modelo (nuevo o el previo si era mejor)
    save_estimator(estimator)


# Ejecutar el entrenamiento de regresión lineal
train_linear_regression()

c:\Users\ASUS\Documents\AAprogramación general\AANacho\Predictiva\PRE-09-selectkbest-para-regresion-dlion7002\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.7.0 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ASUS\Documents\AAprogramación general\AANacho\Predictiva\PRE-09-selectkbest-para-regresion-dlion7002\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.7.0 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ASUS\Documents\AAp

In [8]:
def eval_metrics(y_true, y_pred):
    """
    Función para calcular métricas de evaluación para modelos de regresión.
    
    Calcula tres métricas principales para evaluar el rendimiento:
    - MSE: Error Cuadrático Medio
    - MAE: Error Absoluto Medio  
    - R²: Coeficiente de Determinación
    
    Args:
        y_true: Valores reales de la variable objetivo
        y_pred: Valores predichos por el modelo
    
    Returns:
        tuple: (mse, mae, r2) métricas calculadas
    """
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

    # MSE (Mean Squared Error): Promedio de los errores al cuadrado
    # Penaliza más los errores grandes debido al cuadrado
    # Valores más bajos indican mejor rendimiento
    mse = mean_squared_error(y_true, y_pred)
    
    # MAE (Mean Absolute Error): Promedio de los valores absolutos de los errores
    # Más robusto a outliers que MSE
    # Interpretación directa: error promedio en las mismas unidades que la variable objetivo
    mae = mean_absolute_error(y_true, y_pred)
    
    # R² (R-squared): Coeficiente de determinación
    # Indica qué proporción de la varianza es explicada por el modelo
    # Rango: (-∞, 1], donde 1 es perfecto y 0 significa que el modelo no es mejor que la media
    r2 = r2_score(y_true, y_pred)

    return mse, mae, r2

In [9]:
def report(estimator, mse, mae, r2):
    """
    Función para mostrar un reporte formateado de las métricas del modelo.
    
    Imprime de manera organizada:
    - El tipo de estimador/modelo usado
    - Las tres métricas principales de evaluación
    
    Args:
        estimator: El modelo evaluado
        mse: Error Cuadrático Medio
        mae: Error Absoluto Medio
        r2: Coeficiente de Determinación (R²)
    """
    # Mostrar el tipo de estimador (ej: LinearRegression, MLPRegressor, etc.)
    print(estimator, ":", sep="")
    
    # Mostrar las métricas con formato consistente
    print(f"  MSE: {mse}")  # Error Cuadrático Medio
    print(f"  MAE: {mae}")  # Error Absoluto Medio  
    print(f"   R2: {r2}")   # Coeficiente de Determinación

In [10]:
def check_estimator():
    """
    Función para evaluar el rendimiento del modelo guardado.
    
    Esta función:
    1. Carga los datos de prueba
    2. Carga el modelo guardado
    3. Hace predicciones
    4. Calcula métricas de evaluación
    5. Muestra un reporte del rendimiento
    """
    import pickle

    import pandas as pd
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

    # PASO 1: Cargar y dividir los datos de la misma manera que en entrenamiento
    # Es crucial usar la misma división para obtener resultados comparables
    data, target = load_data()

    x_train, x_test, y_train_true, y_test_true = make_train_test_split(
        x=data,
        y=target,
    )

    # PASO 2: Cargar el modelo previamente entrenado y guardado
    estimator = load_estimator()

    # PASO 3: Calcular métricas de evaluación
    # Usar el conjunto de prueba para evaluar el rendimiento real del modelo
    mse, mae, r2 = eval_metrics(
        y_test_true,                    # Valores reales del conjunto de prueba
        estimator.predict(x_test),      # Predicciones del modelo en el conjunto de prueba
    )

    # PASO 4: Mostrar reporte del rendimiento
    # estimator.best_estimator_ contiene el mejor modelo encontrado por GridSearchCV
    report(estimator.best_estimator_, mse, mae, r2)


# Ejecutar la evaluación del modelo guardado
check_estimator()

Pipeline(steps=[('tranformer',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('ohe',
                                                  OneHotEncoder(dtype='int'),
                                                  ['Origin'])])),
                ('selectkbest',
                 SelectKBest(k=6,
                             score_func=<function f_regression at 0x000001F1F9ACC2C0>)),
                ('estimator', LinearRegression())]):
  MSE: 11.177256954645403
  MAE: 2.524517976430189
   R2: 0.8089190863428402


In [ ]:
def train_mlp_regressor():
    """
    Función para entrenar un modelo de Red Neuronal Multicapa (MLP) para regresión.
    
    Similar a train_linear_regression(), pero usa MLPRegressor y busca
    hiperparámetros adicionales específicos de redes neuronales:
    - Número de características (k)
    - Tamaño de capas ocultas
    - Tasa de aprendizaje
    
    MLP es más complejo que regresión lineal y puede capturar relaciones no lineales.
    """
    from sklearn.neural_network import MLPRegressor
    from sklearn.metrics import mean_absolute_error

    # PASO 1: Cargar y preparar los datos (igual que en regresión lineal)
    data, target = load_data()

    # PASO 2: Dividir en entrenamiento y prueba
    x_train, x_test, y_train, y_test = make_train_test_split(
        x=data,
        y=target,
    )

    # PASO 3: Crear pipeline con MLPRegressor
    # max_iter=30000: número máximo de iteraciones para convergencia
    # MLPs requieren más iteraciones que modelos lineales
    pipeline = make_pipeline(
        estimator=MLPRegressor(max_iter=30000),
    )

    # PASO 4: Configurar búsqueda exhaustiva de hiperparámetros para redes neuronales
    estimator = make_grid_search(
        estimator=pipeline,
        param_grid={
            # Número de características a seleccionar (igual que regresión lineal)
            "selectkbest__k": range(1, len(x_train.columns) + 1),
            
            # Arquitectura de la red: probar redes con 1 a 10 neuronas en una capa oculta
            # (n,) significa una sola capa oculta con n neuronas
            "estimator__hidden_layer_sizes": [(n,) for n in range(1, 11)],
            
            # Algoritmo de optimización: Adam es eficiente para la mayoría de problemas
            "estimator__solver": ["adam"],
            
            # Tasa de aprendizaje: controla qué tan grandes son los pasos de actualización
            # Valores más pequeños = aprendizaje más lento pero potencialmente más estable
            "estimator__learning_rate_init": [0.01, 0.001, 0.0001],
        },
        cv=5,  # Validación cruzada de 5 folds
    )

    # PASO 5: Entrenar con búsqueda de hiperparámetros
    # Esto puede tomar mucho tiempo debido a la complejidad del espacio de búsqueda
    estimator.fit(x_train, y_train)

    # PASO 6: Comparar con el mejor modelo previo (igual lógica que regresión lineal)
    best_estimator = load_estimator()

    if best_estimator is not None:
        # Evaluar modelo guardado previamente
        saved_mae = mean_absolute_error(
            y_true=y_test, y_pred=best_estimator.predict(x_test)
        )

        # Evaluar modelo recién entrenado
        current_mae = mean_absolute_error(
            y_true=y_test, y_pred=estimator.predict(x_test)
        )

        # Conservar el mejor modelo (puede ser el anterior si era superior)
        if saved_mae < current_mae:
            estimator = best_estimator

    # PASO 7: Guardar el mejor modelo
    save_estimator(estimator)


# Ejecutar entrenamiento de red neuronal
train_mlp_regressor()

# Evaluar el modelo final (que puede ser regresión lineal o MLP, el que tenga mejor rendimiento)
check_estimator()

